In [1]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
import pandas as pd
import sys

sys.path.append(os.getcwd())
import tsa_utils

from shutil import copytree
import numpy as np
import matplotlib as mpl

import sklearn
from sklearn import metrics

import pip

from sklearn.model_selection import train_test_split
pip.main(['install', '--upgrade', 'dask'])
pip.main(['install', 'objgraph'])

import objgraph

The directory '/Users/jasminefeldmann/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/jasminefeldmann/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


  Found existing installation: dask 0.13.0
    Uninstalling dask-0.13.0:
      Successfully uninstalled dask-0.13.0


The directory '/Users/jasminefeldmann/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/jasminefeldmann/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


  Running setup.py install for objgraph: started
    Running setup.py install for objgraph: finished with status 'done'


In [2]:
# copytree("/models/multi-class8/multi_class", "/output/multi_class")

In [6]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=40
KERNEL_SIZE1=(1,3,3)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(1,3,3)
POOLSIZE2=(1,3,3)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1,1)
STEPS=3000
XSIZE=256
#XSIZE=512
#XSIZE=270
#YSIZE=340
YSIZE=330
#YSIZE=660
LEARNING_RATE=0.0001
EPSILON=0
IMAGE_DEPTH=16
CHANNELS=1
FLAT_POOL_SIZE=793600

DATA_PATH="../tsa_data/images/"
CHECKPOINT_PATH="/output/multi_class"
MODEL_ID="multi_class9"

tf.logging.set_verbosity(tf.logging.INFO)

slice_locations = {1:("top","right"),
	2:("middle","right"), 
	3:("top","left"),
	4:("middle","left"),
	5:("top","middle"),
	6:("top","right"),
	7:("top","left"),
	8:("middle","right"),
	9:("middle","middle"),
	10:("middle","left"),
	11:("bottom","right"),
	12:("bottom","left"),
	13:("bottom","right"),
	14:("bottom","left"),
	15:("bottom","right"),
	16:("bottom","left"),
	17:("top","middle")}

In [7]:
class ZoneModel():

    def __init__(self, model_id, ids, x_slice, y_slice, data_path, labels, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids
        self.x_slice = x_slice
        self.y_slice = y_slice
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        print(XSIZE, YSIZE)
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        training = (mode==tf.contrib.learn.ModeKeys.TRAIN)
        print(training)
        dropout = tf.layers.dropout(data, rate=0.3, training=training)
        conv1 = tf.layers.conv3d(inputs=data, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", 
                strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1", trainable=not self.localize)
        print(conv1)
        #pool0 = tf.layers.max_pooling3d(inputs=conv1, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool0")
        #print(pool0)
        conv2 = tf.layers.conv3d(inputs=conv1, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same",
                strides=(1, 2, 2), name="conv2", activation=tf.nn.relu, trainable=not self.localize)
        print(conv2)
        pool1 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        print(pool1)
        conv3 = tf.layers.conv3d(inputs=pool1, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same", 
                strides=(DEPTHSTRIDE,2,2), name="conv3", trainable=not self.localize)
        print(conv3)
        pool2 = tf.layers.max_pooling3d(inputs=conv3, pool_size=(1,2,2), strides=POOL_STRIDES, name="pool1")
        print(pool2)
        conv4 = tf.layers.conv3d(inputs=pool2, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same", 
                strides=(1, 1, 1), name="conv4", activation=tf.nn.relu, trainable=not self.localize)
        print(conv4)
        pool3 = tf.layers.max_pooling3d(inputs=conv4, pool_size=POOLSIZE2, strides=POOL_STRIDES, name="pool2", padding="same")
        print(pool3)
        flat_pool = tf.reshape(pool3, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=17, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,17])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.sigmoid(
              logits, name="prob_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            #flat_labels = tf.reshape(labels, [BATCH_SIZE, 17])
            labels=tf.identity(labels, name="labels")
            labels = tf.cast(labels, tf.float32)
            #class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            #print(class_weights)
            #loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
            loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            #x=tsa_utils.ThresholdedInputImagesIterator(self.ids, self.data_path, 10000, self.y_slice, self.x_slice, pool_size=2), 
            x=tsa_utils.InputImagesIterator(self.ids, self.data_path, 10000, self.y_slice, self.x_slice, pool_size=2),
            y=tsa_utils.InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        print(self.checkpoint_path, self.model_id)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=tsa_utils.InputImagesIterator(self.ids, 
                                                            self.data_path, 
                                                            10000, 
                                                            self.y_slice,
                                                            self.x_slice, False, pool_size=2))

In [18]:
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_Zone", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
train_ids, test_ids, _, _ = train_test_split(ids, ids, test_size=.1)
train_ids.sort()
test_ids.sort()
train_labels = image_df[image_df['id'].isin(train_ids)]
train_labels = train_labels.sort_values("id")

tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
train_labels.zone = train_labels.zone.astype("int")
train_labels = train_labels.sort_values(["id","zone"])
train_labels = train_labels["Probability"]
train_labels = np.reshape(train_labels, (-1,17))

test_labels = image_df[image_df['id'].isin(test_ids)]
test_labels = test_labels.sort_values("id")

tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
test_labels.zone = test_labels.zone.astype("int")
test_labels = test_labels.sort_values(["id","zone"])
test_df = test_labels.sort_values(["id","zone"])
test_labels = test_labels["Probability"]
# test_labels = np.reshape(test_labels, (-1,17))

/Users/jasminefeldmann/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:224: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return reshape(newshape, order=order)


In [19]:
test_labels

187      1
196      0
197      0
198      0
199      0
200      0
201      1
202      0
203      0
188      0
189      0
190      1
191      0
192      0
193      0
194      0
195      0
272      0
281      0
282      0
283      0
284      0
285      0
286      0
287      0
288      0
273      1
274      0
275      0
276      0
        ..
19307    1
19308    0
19309    0
19310    1
19311    0
19296    0
19297    0
19298    0
19299    0
19300    1
19301    0
19302    0
19303    0
19414    1
19423    0
19424    0
19425    0
19426    0
19427    0
19428    0
19429    0
19430    0
19415    0
19416    0
19417    0
19418    0
19419    1
19420    0
19421    1
19422    0
Name: Probability, dtype: int64

In [20]:
test_df

,Id,Probability,zone,id
187,038d648c2f29cb0f945c865be25e32e9_Zone1,1,1,038d648c2f29cb0f945c865be25e32e9
196,038d648c2f29cb0f945c865be25e32e9_Zone2,0,2,038d648c2f29cb0f945c865be25e32e9
197,038d648c2f29cb0f945c865be25e32e9_Zone3,0,3,038d648c2f29cb0f945c865be25e32e9
198,038d648c2f29cb0f945c865be25e32e9_Zone4,0,4,038d648c2f29cb0f945c865be25e32e9
199,038d648c2f29cb0f945c865be25e32e9_Zone5,0,5,038d648c2f29cb0f945c865be25e32e9
200,038d648c2f29cb0f945c865be25e32e9_Zone6,0,6,038d648c2f29cb0f945c865be25e32e9
201,038d648c2f29cb0f945c865be25e32e9_Zone7,1,7,038d648c2f29cb0f945c865be25e32e9
202,038d648c2f29cb0f945c865be25e32e9_Zone8,0,8,038d648c2f29cb0f945c865be25e32e9
203,038d648c2f29cb0f945c865be25e32e9_Zone9,0,9,038d648c2f29cb0f945c865be25e32e9
188,038d648c2f29cb0f945c865be25e32e9_Zone10,0,10,038d648c2f29cb0f945c865be25e32e9


In [ ]:
with tf.Session() as sess:
    model = ZoneModel(MODEL_ID, train_ids, "trimmed", "both", DATA_PATH, train_labels, CHECKPOINT_PATH)     
    model.train_model(tensors_to_log, reuse=False)

INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/output/multi_class/multi_class9', '_master': '', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_tf_random_seed': None, '_evaluation_master': '', '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_is_chief': True, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc969296f60>, '_keep_checkpoint_max': 5, '_task_id': 0}
Using config: {'_model_dir': '/output/multi_class/multi_class9', '_master': '', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_tf_random_seed': None, '_evaluation_master': '', '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_

From <ipython-input-11-0aef3610bdd7>:85: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with batch_size is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-11-0aef3610bdd7>:85: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with y is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-11-0aef3610bdd7>:85: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


256 330
True
Tensor("conv1/BiasAdd:0", shape=(10, 16, 330, 256, 40), dtype=float32)
Tensor("conv2/Relu:0", shape=(10, 16, 165, 128, 40), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(10, 16, 82, 63, 40), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(10, 16, 41, 32, 40), dtype=float32)
Tensor("pool1_2/MaxPool3D:0", shape=(10, 16, 40, 31, 40), dtype=float32)
Tensor("conv4/Relu:0", shape=(10, 16, 40, 31, 40), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(10, 16, 40, 31, 40), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /output/multi_class/multi_class9/model.ckpt.
Saving checkpoints for 1 into /output/multi_class/multi_class9/model.ckpt.
INFO:tensorflow:step = 1, loss = 120.525
step = 1, loss = 120.525
INFO:tensorflow:probabilities = [[ 0.3809956   0.63464999  0.54556298  0.49847746  0.49459207  0.5672037
   0.6102758   0.42079791  0.3599309   0.50141609  0.46498829  0.49663934
   0.38281095  

# Calculate F1

In [70]:

model = ZoneModel(MODEL_ID, test_ids, "both","both", DATA_PATH, test_labels, CHECKPOINT_PATH)
model.load_model()
predicted = np.array([x["probabilities"] for x in list(model.predict())])
predicted[predicted < .4] = 0
predicted[predicted > .4] = 1
predicted = predicted.flatten()

print(sklearn.metrics.f1_score(test_labels, predicted))
print(sklearn.metrics.precision_score(test_labels, predicted))
print(sklearn.metrics.recall_score(test_labels, predicted))



/output/multi_class multi_class8
INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_environment': 'local', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f05e45470>, '_model_dir': '/output/multi_class/multi_class8', '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_session_config': None, '_master': '', '_keep_checkpoint_max': 5, '_is_chief': True, '_num_worker_replicas': 0, '_save_checkpoints_secs': 600, '_task_id': 0, '_task_type': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}
Using config: {'_tf_random_seed': None, '_environment': 'local', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f05e45470>, '_model_dir': '/output/multi_class/multi_class8', '_num_ps_replicas': 0, '_tf_config': gpu_o

From <ipython-input-69-566c6ff91ec6>:109: calling BaseEstimator.predict (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


256 330
False
Tensor("conv1/BiasAdd:0", shape=(1, 16, 330, 256, 22), dtype=float32)
Tensor("conv2/Relu:0", shape=(1, 16, 165, 128, 22), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(1, 16, 82, 63, 22), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(1, 16, 41, 32, 22), dtype=float32)
Tensor("pool1_2/MaxPool3D:0", shape=(1, 16, 40, 31, 22), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 16, 40, 31, 22), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(1, 16, 40, 31, 22), dtype=float32)
INFO:tensorflow:Restoring parameters from /output/multi_class/multi_class8/model.ckpt-4972
Restoring parameters from /output/multi_class/multi_class8/model.ckpt-4972
0.102857142857
0.10843373494
0.0978260869565


In [78]:
eval_df = pd.read_csv(DATA_PATH + 'stage1_sample_submission.csv')
eval_df['zone'] = eval_df['Id'].str.split("_", expand=True)[1].str.strip()
eval_df['id'] = eval_df['Id'].str.split("_", expand=True)[0].str.strip()

eval_ids = eval_df["id"].unique()
eval_ids.sort()

csv_file = open(CHECKPOINT_PATH + "/submission2.csv", "w+")
csv_file.write("Id,Probability\n")

model = ZoneModel(MODEL_ID, eval_ids, "both","both", DATA_PATH, test_labels, CHECKPOINT_PATH)
model.load_model()
print ("LOADED MODEL " + str(model))
predicted = np.array([x["probabilities"] for x in list(model.predict())])
for i, image in enumerate(predicted):
    for j, zone in enumerate(image):
        csv_file.write(str(eval_ids[i]) + "_Zone" + str(j+1) + "," + str(zone) + "\n")
csv_file.close()



/output/multi_class multi_class8
INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_environment': 'local', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f0402ff60>, '_model_dir': '/output/multi_class/multi_class8', '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_session_config': None, '_master': '', '_keep_checkpoint_max': 5, '_is_chief': True, '_num_worker_replicas': 0, '_save_checkpoints_secs': 600, '_task_id': 0, '_task_type': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}
Using config: {'_tf_random_seed': None, '_environment': 'local', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f0402ff60>, '_model_dir': '/output/multi_class/multi_class8', '_num_ps_replicas': 0, '_tf_config': gpu_o

From <ipython-input-69-566c6ff91ec6>:109: calling BaseEstimator.predict (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


256 330
False
Tensor("conv1/BiasAdd:0", shape=(1, 16, 330, 256, 22), dtype=float32)
Tensor("conv2/Relu:0", shape=(1, 16, 165, 128, 22), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(1, 16, 82, 63, 22), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(1, 16, 41, 32, 22), dtype=float32)
Tensor("pool1_2/MaxPool3D:0", shape=(1, 16, 40, 31, 22), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 16, 40, 31, 22), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(1, 16, 40, 31, 22), dtype=float32)
INFO:tensorflow:Restoring parameters from /output/multi_class/multi_class8/model.ckpt-4972
Restoring parameters from /output/multi_class/multi_class8/model.ckpt-4972


In [77]:
eval_ids

array(['0367394485447c1c3485359ba71f52cb',
       '0749d03b6d2d1eebb6c24ecb3c36e645',
       '08dc52d08482393070d8c71b11902636',
       '0a83698bce92a6824dcc37c1d7fc31f5',
       '0b2996d90c7250a3acb63cab6ae5bebe',
       '0fadb2ac7be1a0ed6fb7cd81372c18ba',
       '0fd54393dce1f55b5a4d7e30b8642158',
       '104e284349ffe68378745fc8c5638eab',
       '123d65c19a4d7f395b3719d47d5666b5',
       '1240199d7b5d6d1c709b13381a198b75',
       '12ed0db15024397106482c4983ad2d92',
       '1b861c23fc370c326ec8342733ef5d84',
       '1cb13f156bd436222447dd658180bd96',
       '1ccf082d6f3d51bd55113fb092be0e59',
       '1cef83028272a04070fbdffae5796a87',
       '249092cc1775fabcb79ba439ba974759',
       '27069e6977eb7e0ccd913af254932e39',
       '28970155f9954bbd7d5c737b60a66b5a',
       '2b81524a1a36b6b91baf7c84dc16b29b',
       '334cfc11aceab7e8bbd1d711f70c514d',
       '35fcc4eea20865907dd58ad98684d52b',
       '366b47c7a8ccab2cd0e5834dde998765',
       '3de88afea1b8fd356b119c5b44dcb47e',
       '433

In [9]:
len(test_labels)

115

In [10]:
len(test_ids)

115